In [1]:
!wget https://uwnlp.github.io/storycommonsense/data/storycommonsense_data.zip -O cache_data/storycommonsense_data.zip

--2023-10-08 20:50:19--  https://uwnlp.github.io/storycommonsense/data/storycommonsense_data.zip
Resolving uwnlp.github.io (uwnlp.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to uwnlp.github.io (uwnlp.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18755402 (18M) [application/zip]
Saving to: ‘cache_data/storycommonsense_data.zip’

cache_data/storycom 100%[===================>]  17.89M  9.78MB/s    in 1.8s    

2023-10-08 20:50:21 (9.78 MB/s) - ‘cache_data/storycommonsense_data.zip’ saved [18755402/18755402]



In [5]:
!unzip cache_data/storycommonsense_data.zip -d cache_data/storycommonsense_data

Archive:  cache_data/storycommonsense_data.zip
   creating: cache_data/storycommonsense_data/csv_version/
   creating: cache_data/storycommonsense_data/csv_version/dev/


   creating: cache_data/storycommonsense_data/csv_version/dev/emotion/
  inflating: cache_data/storycommonsense_data/csv_version/dev/emotion/allcharlinepairs.csv  
   creating: cache_data/storycommonsense_data/csv_version/dev/motiv/
  inflating: cache_data/storycommonsense_data/csv_version/dev/motiv/allcharlinepairs.csv  
  inflating: cache_data/storycommonsense_data/csv_version/README.txt  
   creating: cache_data/storycommonsense_data/csv_version/test/
   creating: cache_data/storycommonsense_data/csv_version/test/emotion/
  inflating: cache_data/storycommonsense_data/csv_version/test/emotion/allcharlinepairs.csv  
   creating: cache_data/storycommonsense_data/csv_version/test/motiv/
  inflating: cache_data/storycommonsense_data/csv_version/test/motiv/allcharlinepairs.csv  
   creating: cache_data/storycommonsense_data/csv_version/training/
  inflating: cache_data/storycommonsense_data/csv_version/training/allcharlinepairs.csv  
   creating: cache_data/storycommonsense_data/csv_versi

In [40]:
import json
from textwrap import dedent

from datasets import Dataset
from huggingface_hub import notebook_login
from tqdm.auto import tqdm


In [11]:
with open('cache_data/storycommonsense_data/json_version/annotations.json') as f:
    data = json.load(f)

In [12]:
data["0008c800-82b6-43b3-8b53-5475ed1dac9b"]

{'lines': {'1': {'characters': {'My daughter': {'app': False,
     'emotion': {},
     'motiv': {}},
    'Nana': {'app': True,
     'emotion': {'ann0': {'plutchik': ['sadness:2', 'disgust:2', 'anger:2'],
       'text': ['confused']},
      'ann1': {'plutchik': ['surprise:2'], 'text': ['confused']}},
     'motiv': {'ann0': {'maslow': ['stability'],
       'reiss': ['order'],
       'text': ['to understand']},
      'ann1': {'maslow': ['none'],
       'reiss': [],
       'text': ['to check something']},
      'ann2': {'maslow': ['esteem'],
       'reiss': ['status'],
       'text': ['to be noticed']}}}},
   'text': 'Nana came into the room with a puzzled look on her face.'},
  '2': {'characters': {'My daughter': {'app': False,
     'emotion': {},
     'motiv': {}},
    'Nana': {'app': True,
     'emotion': {'ann0': {'plutchik': ['surprise:2'], 'text': ['confused']},
      'ann1': {'plutchik': ['surprise:2', 'anger:2'],
       'text': ['confused', 'frustrated']},
      'ann2': {'plutchik'

In [32]:
processed_data = [
    dict(
        title=record["title"],
        story="\n".join([
            line["text"] for line in record["lines"].values()
        ]),
        characters=set([
            char
            for line in record["lines"].values()
            for char in line["characters"].keys()
        ]),
        emotions={
            char: [
                {
                    "emotion": [
                        emotion_data
                        for emotion_data in char_data["emotion"].values()
                        if '3' in ''.join(emotion_data.get("plutchik", []))
                    ],
                    "line_no": line_no,
                }
                for line_no, line in record["lines"].items()
                for char_name, char_data in line["characters"].items()
                if char_name == char and char_data["emotion"]
            ]
            for line in record["lines"].values()
            for char in line["characters"].keys()
        },
    )
    for record in tqdm(data.values())
]

  0%|          | 0/14738 [00:00<?, ?it/s]

In [34]:
notebook_login()

In [36]:
dataset = Dataset.from_list(processed_data)

In [37]:
dataset.push_to_hub("diwank/storycommonsense-raw")

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/15 [00:00<?, ?ba/s]

In [38]:
make_chatml = lambda name, role, content: dict(
    name=name, role=role, content=content,
)

system = lambda name, content: make_chatml(
    role="system",
    name=name,
    content=content,
)

situation = lambda content: system(name="situation", content=content)
thought = lambda content: system(name="thought", content=content)
information = lambda content: system(name="information", content=content)
me = lambda content, name=None: make_chatml(
    role="assistant",
    content=content,
    name=name,
)

person = lambda content, name=None: make_chatml(
    role="user",
    content=content,
    name=name,
)


In [42]:
def to_chatml(row):
    title = row["title"]
    story = row["story"]
    characters = row["characters"]
    emotions = row["emotions"]

    situation_content = dedent(f"""\
    Hello
    """).strip()

    chatml = [
        situation(content=situation_content),
    ]

    return dict(chatml=chatml)

to_chatml(processed_data[0])
processed_data[0]

{'title': 'Sock Chase',
 'story': 'Nana came into the room with a puzzled look on her face.\nShe held up an orange sock and a blue one.\nMy daughter jumped up and grabbed the blue one out of her hand.\nShe took off running down the hall while waving the sock in the air.\nNana chased her down, caught her, and tickled her until she laughed.',
 'characters': {'My daughter', 'Nana'},
 'emotions': {'My daughter': [{'emotion': [], 'line_no': '3'},
   {'emotion': [{'plutchik': ['joy:3'], 'text': ['happy']}], 'line_no': '4'},
   {'emotion': [{'plutchik': ['joy:3',
       'trust:3',
       'surprise:3',
       'anticipation:2'],
      'text': ['ecstatic']},
     {'plutchik': ['joy:3', 'trust:3', 'surprise:3', 'anticipation:3'],
      'text': ['fun']}],
    'line_no': '5'}],
  'Nana': [{'emotion': [], 'line_no': '1'},
   {'emotion': [], 'line_no': '2'},
   {'emotion': [{'plutchik': ['joy:3', 'surprise:2'], 'text': ['amused']}],
    'line_no': '3'},
   {'emotion': [{'plutchik': ['joy:3', 'trust:2